In [1]:
!pip install datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e7b08ee73d19ac2c52f02d564f69571d842f979e4b628bd96bf82e65939d6d53
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's depe

In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    TrainerCallback,
)
from datasets import load_dataset
from tqdm import tqdm
import evaluate


In [3]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

train_data = load_dataset("cnn_dailymail", "3.0.0", split="train[:5000]")
val_data = load_dataset("cnn_dailymail", "3.0.0", split="validation[:1000]")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [15]:
def preprocess_function(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=1024)
    outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=512)

    # batch["input_ids"] = inputs.input_ids
    # batch["attention_mask"] = inputs.attention_mask
    # batch["labels"] = [
    #     [(token if token != tokenizer.pad_token_id else -100) for token in label]
    #     for label in outputs.input_ids
    # ]
    # return batch

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': outputs['input_ids']
    }


In [5]:
train_data = train_data.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])
val_data = val_data.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
training_args = Seq2SeqTrainingArguments(
    output_dir="flan-t5_checkpoints",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs = 5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    load_best_model_at_end=True,
    eval_steps=100,
    generation_max_length=128,
    predict_with_generate=True,
    logging_steps=100,
    gradient_accumulation_steps=1,
    fp16=True,
    remove_unused_columns=False,
    report_to="none",
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
metric = evaluate.load("rouge")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions

    if preds.max() >= len(tokenizer):
      print("Warning: Some prediction token IDs are out of range.")
      print(preds)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    scores = metric.compute(predictions=decoded_preds, references=decoded_labels, rouge_types=["rouge1"])
    return scores


In [8]:
class TQDMProgressBar(TrainerCallback):
    def __init__(self):
        self.pbar = None


    def on_train_begin(self, args, state, control, **kwargs):
        self.pbar = tqdm(total=args.max_steps, desc="Training Progress", unit="step")

    def on_step_end(self, args, state, control, **kwargs):
        self.pbar.update(1)
        if state.log_history:
            latest_log = state.log_history[-1]
            desc = ", ".join([f"{k}: {v:.4f}" for k, v in latest_log.items() if isinstance(v, (int, float))])
            self.pbar.set_postfix_str(desc)

    def on_train_end(self, args, state, control, **kwargs):
        self.pbar.close()

progress_bar = TQDMProgressBar()

In [9]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[progress_bar],
)


<ipython-input-9-6d4a68731f9a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [16]:
trainer.train()

# results = trainer.evaluate()
# print(results)

Training Progress: 313step [12:35,  2.41s/step, loss: 1.1345, grad_norm: 0.7844, learning_rate: 0.0000, epoch: 0.9585, step: 300.0000]
Training Progress: 1step [00:00,  2.48step/s]

Epoch,Training Loss,Validation Loss


Training Progress: 313step [01:57,  3.17step/s, loss: 1.1111, grad_norm: 0.7096, learning_rate: 0.0000, epoch: 0.9585, step: 300.0000]

Epoch,Training Loss,Validation Loss


OverflowError: out of range integral type conversion attempted

In [13]:
tokenizer.decode(-100)

OverflowError: out of range integral type conversion attempted